In [ ]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from elfpy.simulators import Simulator
from elfpy.utils.post_processing import analysis_dict_to_dataframe
from elfpy.utils import sim_utils  # utilities for setting up a simulation
import elfpy.utils.parse_config as config_utils
import elfpy.utils.outputs as output_utils

In [ ]:
pd.set_option("float_format",'{:,.3f}'.format)

In [ ]:
start_time = time.time()

config_file = "../../config/example_config.toml"
override_dict = {
    "pricing_model_name": "YieldSpace",
}
# get config & logging level
config = sim_utils.override_config_variables(config_utils.load_and_parse_config_file(config_file), override_dict)
# define root logging parameters
output_utils.setup_logging(
    log_filename="../../.logging/sim_one_model.log", log_level=config.simulator.logging_level
)
# instantiate random number generator
rng = np.random.default_rng(config.simulator.random_seed)
pricing_model = sim_utils.get_pricing_model(model_name=config.amm.pricing_model_name)
simulator = sim_utils.get_simulator(config, rng, pricing_model, [])

num_runs = 1 # lots of runs to have high statistical power
for run_number in range(num_runs): # run a simulation per pricing model
    override_dict = {
        "pricing_model_name": "YieldSpace",
        "num_trading_days": 3,
        "num_blocks_per_day": 5,
        #"vault_apr": [0.1,]*config["num_trading_days"],
        #"init_vault_age": 1,
        #"target_pool_apy" : 0.05,
        #"target_liquidity": 3e6,
        #"fee_percent" : 0.1,
    }
    simulator.random_variables = sim_utils.get_random_variables(config, rng) # reset random variables
    simulator.config = sim_utils.override_config_variables(config, override_dict)
    simulator.run_simulation()

end_time = time.time()
print(f'Total time for {num_runs} runs was {end_time-start_time:.3f} seconds; which is {(end_time-start_time)/num_runs:.3f} seconds per run')

In [ ]:
trades = analysis_dict_to_dataframe(simulator.analysis_dict)

In [ ]:
print(f'Simulator contains {trades.shape[1]} variables tracked over {trades.shape[0]} simulations.')
print(f'Trades dataframe variables:\n'+'\t'.join(list(trades.columns)))
print(f'\nTrades dataframe:')
display(pd.concat([trades.head(4), trades.tail(2)]).T)